In [2]:
import lsst.daf.butler as dafButler
import numpy as np
import glob
import os, sys

## From Shany:

In [5]:

output_collection = "DECam/runs/merian/dr1_wide"
data_type = "deepCoadd_calexp"
skymap = "hsc_rings_v1"
butler = dafButler.Butler('/projects/MERIAN/repo/', collections=output_collection, skymap=skymap)


In [6]:

patches = np.array([[data_id['tract'], data_id["patch"]] for data_id in butler.registry.queryDataIds (['tract','patch'], datasets=data_type, 
                                                 collections=output_collection, skymap=skymap)])
patches = patches[patches[:, 0].argsort()]

In [7]:
tracts, idx = np.unique(patches[:,0], return_index=True) 
patches_by_tract = np.split(patches[:,1] ,idx[1:])

In [7]:
# f = open("reducedPatches.csv", "w")
# for i, tract in enumerate(tracts):
#     f.write(f'{tract}')

#     patches_i = np.unique(patches_by_tract[i])
#     patches_i.sort()
#     for p in patches_i:
#         f.write(f',{p}')
#     f.write("\n")

# f.close()


---

## Used in `gaap`:

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
merian_patches = [item.dataId for item in butler.registry.queryDatasets('objectTable', 
                        dataId=dict(tract=9813), collections=output_collection,
                        skymap=skymap)]

In [10]:
np.unique(merian_patches)

array([GovernorDimension(skymap), DatabaseDimension(tract),
       DatabaseDimension(patch)], dtype=object)

In [11]:
butler = dafButler.Butler('/projects/MERIAN/repo/')

[item.dataId for item in butler.registry.queryDatasets('objectTable', 
                        dataId=dict(tract=9813, patch=3), collections='DECam/runs/merian/dr1_wide',
                        skymap='hsc_rings_v1')]

[]

In [12]:
IDs = butler.registry.queryDataIds(['tract','patch'], datasets=data_type, 
                                            collections=output_collection, skymap=skymap)

In [13]:
for id in butler.registry.queryDataIds(['tract','patch'], datasets=data_type, 
                                            collections=output_collection, skymap=skymap):
    if (id["tract"] == 9813) & (id["patch"] == 3):
        print(id)

{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}


In [14]:
for data_id in butler.registry.queryDataIds(['tract','patch'], datasets=data_type, 
                                            collections=output_collection, skymap=skymap):
    print(data_id)

{skymap: 'hsc_rings_v1', tract: 9085, patch: 49}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 50}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 46}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 49}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 50}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 62}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 61}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 45}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 66}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 80}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 60}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 55}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 77}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 58}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 51}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 65}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 54}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 74}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 57}
{skymap: 'hsc_rings_v1', tract: 9085, patch: 56}
{skymap: 'hsc_rings_

In [15]:
tract=8282

In [15]:
output_collection = "DECam/runs/merian/dr1_wide"
data_type = "deepCoadd_calexp"
skymap = "hsc_rings_v1"

butler = dafButler.Butler('/projects/MERIAN/repo/', collections=output_collection, skymap=skymap)

patches = np.array([[data_id['tract'], data_id["patch"]] for data_id in butler.registry.queryDataIds (['tract','patch'], datasets=data_type, 
                                                collections=output_collection, skymap=skymap)])
patches = patches[patches[:, 0].argsort()]
del butler 

patches = np.unique(patches[patches[:,0] == tract][:,1])


NameError: name 'tract' is not defined

In [ ]:
output_collection = "DECam/runs/merian/dr1_wide"
data_type = "deepCoadd_calexp"
skymap = "hsc_rings_v1"

butler = dafButler.Butler('/projects/MERIAN/repo/', collections=output_collection, skymap=skymap)

patches = np.array([[data_id['tract'], data_id["patch"]] for data_id in butler.registry.queryDataIds (['tract','patch'], datasets=data_type, 
                                                collections=output_collection, skymap=skymap)])
patches = patches[patches[:, 0].argsort()]
# del butler 

patches = np.unique(patches[patches[:,0] == tract][:,1])

----
There is a problem where not every patch returned by the codes above have measurements in N708. Specifically, they don't have `deepCoadd_meas` datasets with data ID (for example): `{band:'N708', skymap:'hsc_rings_v1', tract:8279, patch:54}`.

So if I want to get a list of only patches that have N708, use this code:

----
# Abby's solution:

Okay actually, I need every patch to have the following four data products:

- deepCoadd_ref
- deepCoadd_meas
- deepCoadd_scarletModelData
- deepCoadd_calexp

So I don't actually need an `objecTable`. Let's see if this works.

In [16]:
def findReducedPatches(tract, band="N708", output_collection='DECam/runs/merian/dr1_wide', skymap='hsc_rings_v1'):
        butler = dafButler.Butler('/projects/MERIAN/repo/')

        dataId = dict(tract=tract, band=band, skymap=skymap)
        
        deepCoadd_ref_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_ref', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_meas_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_meas', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_scarletModelData_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_scarletModelData', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_calexp_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_calexp', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        del butler

        patches = deepCoadd_ref_patches.intersection(deepCoadd_meas_patches, deepCoadd_scarletModelData_patches, deepCoadd_calexp_patches)
        return(np.array(list(patches)))

In [17]:
def findGaapReducedPatches(tract, repo = '/scratch/gpfs/am2907/Merian/gaap/'):
    dir = os.path.join(repo, f"log/{tract}/")
    logs = np.array(glob.glob(dir + "*.o"))
    logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
    logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]

    return (logs_patches)

In [18]:
tracts_n708 = []
for tract in tracts:
    patches = ",".join(findReducedPatches(tract).astype(str))
    if len(patches) > 0:
        tracts_n708.append(tract)

In [19]:
tracts_n708 = np.array(tracts_n708)
len(tracts_n708)

285

In [21]:
tracts_n708

array([ 8280,  8281,  8282,  8283,  8284,  8285,  8286,  8521,  8522,
        8523,  8524,  8525,  8526,  8527,  8764,  8765,  8766,  8767,
        8768,  8769,  9010,  9011,  9071,  9072,  9073,  9074,  9075,
        9076,  9077,  9078,  9079,  9080,  9081,  9082,  9083,  9084,
        9085,  9086,  9087,  9088,  9089,  9090,  9091,  9092,  9096,
        9097,  9098,  9099,  9100,  9101,  9102,  9103,  9104,  9131,
        9132,  9133,  9134,  9135,  9136,  9226,  9227,  9313,  9314,
        9315,  9316,  9317,  9318,  9319,  9320,  9321,  9322,  9323,
        9324,  9325,  9326,  9327,  9328,  9329,  9330,  9331,  9332,
        9333,  9334,  9335,  9336,  9339,  9340,  9341,  9342,  9343,
        9344,  9345,  9346,  9347,  9373,  9374,  9375,  9376,  9377,
        9378,  9379,  9455,  9456,  9457,  9458,  9459,  9465,  9466,
        9467,  9468,  9469,  9470,  9471,  9556,  9557,  9558,  9559,
        9560,  9561,  9562,  9563,  9564,  9565,  9566,  9567,  9568,
        9569,  9570,

In [22]:
# for tract in tracts_n708:
#     patches = findReducedPatches(tract)
#     if len(patches) > 0:
#         print (f'TRACT:{tract}, {len(patches)}')


TRACT:8280, 3
TRACT:8281, 29
TRACT:8282, 30
TRACT:8283, 36
TRACT:8284, 40
TRACT:8285, 34
TRACT:8286, 1
TRACT:8521, 8
TRACT:8522, 73
TRACT:8523, 73
TRACT:8524, 78
TRACT:8525, 81
TRACT:8526, 81
TRACT:8527, 28
TRACT:8764, 6
TRACT:8765, 4
TRACT:8766, 7
TRACT:8767, 27
TRACT:8768, 80
TRACT:8769, 52
TRACT:9010, 8
TRACT:9011, 4
TRACT:9071, 14
TRACT:9072, 18
TRACT:9073, 24
TRACT:9074, 22
TRACT:9075, 25
TRACT:9076, 25
TRACT:9077, 27
TRACT:9078, 27
TRACT:9079, 27
TRACT:9080, 26
TRACT:9081, 26
TRACT:9082, 29
TRACT:9083, 30
TRACT:9084, 32
TRACT:9085, 30
TRACT:9086, 35
TRACT:9087, 36
TRACT:9088, 36
TRACT:9089, 35
TRACT:9090, 35
TRACT:9091, 35
TRACT:9092, 19
TRACT:9096, 6
TRACT:9097, 33
TRACT:9098, 35
TRACT:9099, 32
TRACT:9100, 39
TRACT:9101, 41
TRACT:9102, 35
TRACT:9103, 35
TRACT:9104, 14
TRACT:9131, 12
TRACT:9132, 27
TRACT:9133, 25
TRACT:9134, 27
TRACT:9135, 26
TRACT:9136, 8
TRACT:9226, 4
TRACT:9227, 4
TRACT:9313, 59
TRACT:9314, 81
TRACT:9315, 81
TRACT:9316, 81
TRACT:9317, 81
TRACT:9318, 81
TRACT:9

In [192]:
# f = open("../../../reducedPatches_N708.csv", "w")

# for tract in tracts:
#     patches = ",".join(findReducedPatches(tract).astype(str))
#     if len(patches) > 0:
#         f.write(f'{tract},{patches}\n')
# f.close()



In [196]:
# f = open("../../../reducedTracts_n708.csv", "w")
# f.write("Tract,patches\n")
# for tract in tracts:
#     patches = findReducedPatches(tract)
#     if len(patches) > 0:
#         f.write(f'{tract},{len(patches)}\n')
# f.close()

---

## Investigating for Erin

In [18]:
erintracts= [9378,9379,9454,9455,9456,9457,9458,9459,9460,9461,9462,9463,9464,9465,9466,9467,9468,9469,9470,9471,9491,9492,9493,9494,9495,9496,9556,9557,9558,9559,9560,9561,9562,9563,9564,9565]

In [19]:
band = "N540"
for tract in erintracts:
        dataId = dict(tract=tract, band=band, skymap=skymap)

        deepCoadd_ref_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_ref', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_meas_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_meas', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_scarletModelData_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_scarletModelData', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_calexp_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_calexp', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])


        patches_int = deepCoadd_ref_patches.intersection(deepCoadd_meas_patches, deepCoadd_scarletModelData_patches, deepCoadd_calexp_patches)
        patches_uni = deepCoadd_ref_patches.union(deepCoadd_meas_patches, deepCoadd_scarletModelData_patches, deepCoadd_calexp_patches)
        if len(patches_int) != len(patches_uni):
                print(f"Tract: {tract}")
                print (f"Band: {band}")
                print(f'ref missing: {patches_uni - deepCoadd_ref_patches}')
                print(f'meas missing: {patches_uni - deepCoadd_meas_patches}')
                print(f'scarlet missing: {patches_uni - deepCoadd_scarletModelData_patches}')
                print(f'calexp missing: {patches_uni - deepCoadd_calexp_patches}')
                print()

Tract: 9378
Band: N540
ref missing: {59}
meas missing: set()
scarlet missing: set()
calexp missing: set()

Tract: 9455
Band: N540
ref missing: set()
meas missing: {72, 73, 74, 75, 76, 77, 78, 79}
scarlet missing: set()
calexp missing: {72, 73, 74, 75, 76, 77, 78, 79}

Tract: 9456
Band: N540
ref missing: set()
meas missing: {80, 79}
scarlet missing: set()
calexp missing: {80, 79}

Tract: 9556
Band: N540
ref missing: set()
meas missing: {74}
scarlet missing: set()
calexp missing: {74}

Tract: 9557
Band: N540
ref missing: set()
meas missing: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80}
scarlet missing: set()
calexp missing: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27

In [20]:
skymap_map = butler.get("skyMap", collections="skymaps", skymap="hsc_rings_v1")

def getPatchCenter(skymap, tract, patch):
    """Find the center of the patch BBox in pixel coordinates and convert using tract WCS."""
    wcs = skymap_map.generateTract(tract).getWcs()
    bbox = skymap_map.generateTract(tract).getPatchInfo(patch).getInnerBBox()
    sky = wcs.pixelToSky(
        np.mean((bbox.getBeginX(), bbox.getEndX())),
        np.mean((bbox.getBeginY(), bbox.getEndY()))
    )
    return sky[0].asDegrees(), sky[1].asDegrees()


In [21]:
tract= 9461
print (f"Tract: {tract}")

for patch in range(73,80): 
    print (f"Patch: {patch}")
    ra, dec = getPatchCenter(skymap_map, tract, patch)
    print ('%.8f deg, %.8f deg' %(ra, dec))


Tract: 9461
Patch: 73
348.70803611 deg, 0.00286927 deg
Patch: 74
348.52139784 deg, 0.00286935 deg
Patch: 75
348.33475164 deg, 0.00286939 deg
Patch: 76
348.14810149 deg, 0.00286941 deg
Patch: 77
347.96145133 deg, 0.00286939 deg
Patch: 78
347.77480514 deg, 0.00286935 deg
Patch: 79
347.58816687 deg, 0.00286927 deg


In [22]:
tract= 9461

In [23]:
dataId = dict(tract=tract, band="N708", skymap=skymap)

deepCoadd_ref_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_ref', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

deepCoadd_meas_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_meas', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

deepCoadd_scarletModelData_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_scarletModelData', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

deepCoadd_calexp_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_calexp', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

In [24]:
np.array(list(deepCoadd_ref_patches)), np.array(list(deepCoadd_meas_patches)), np.array(list(deepCoadd_scarletModelData_patches)), np.array(list(deepCoadd_calexp_patches))

(array([73, 74, 75, 76, 77, 78, 79]),
 array([], dtype=float64),
 array([73, 74, 75, 76, 77, 78, 79]),
 array([], dtype=float64))

---

# Want to find patches near and around Cosmos

In [31]:
from astropy.coordinates import SkyCoord

In [25]:
skymap_map = butler.get("skyMap", collections="skymaps", skymap="hsc_rings_v1")

def getPatchCenter(skymap, tract, patch):
    """Find the center of the patch BBox in pixel coordinates and convert using tract WCS."""
    wcs = skymap_map.generateTract(tract).getWcs()
    bbox = skymap_map.generateTract(tract).getPatchInfo(patch).getInnerBBox()
    sky = wcs.pixelToSky(
        np.mean((bbox.getBeginX(), bbox.getEndX())),
        np.mean((bbox.getBeginY(), bbox.getEndY()))
    )
    return sky[0].asDegrees(), sky[1].asDegrees()


In [28]:
cosmos_center = getPatchCenter(skymap_map, 9813, 50)
cosmos_center = SkyCoord(cosmos_center[0], cosmos_center[1], unit="deg")

In [36]:
cosmos_center = SkyCoord(cosmos_center[0], cosmos_center[1], unit="deg")

In [57]:
distance = np.empty(len(tracts_n708), dtype=object)
for i, tract in enumerate(tracts_n708):
    tract_center = getPatchCenter(skymap_map, tract, 50)
    tract_center = SkyCoord(tract_center[0], tract_center[1], unit="deg")
    distance[i] = cosmos_center.separation(tract_center)

In [69]:
distance[np.argsort(distance)]

array([<Angle 0. deg>, <Angle 3.03869692 deg>, <Angle 3.09786106 deg>,
       <Angle 3.64115981 deg>, <Angle 3.78777472 deg>,
       <Angle 3.87630314 deg>, <Angle 4.46278621 deg>,
       <Angle 4.65508752 deg>, <Angle 4.70405578 deg>,
       <Angle 4.70408578 deg>, <Angle 4.84614904 deg>,
       <Angle 5.36315792 deg>, <Angle 5.36321054 deg>,
       <Angle 5.87103589 deg>, <Angle 6.08362645 deg>,
       <Angle 6.12432486 deg>, <Angle 6.31050129 deg>,
       <Angle 6.31056837 deg>, <Angle 7.18719067 deg>,
       <Angle 7.41102458 deg>, <Angle 7.43673524 deg>,
       <Angle 7.43681113 deg>, <Angle 8.15867699 deg>,
       <Angle 8.55743994 deg>, <Angle 8.67244343 deg>,
       <Angle 8.67252478 deg>, <Angle 8.78768914 deg>,
       <Angle 9.61844789 deg>, <Angle 9.9594811 deg>,
       <Angle 9.97703148 deg>, <Angle 9.97711634 deg>,
       <Angle 10.193679 deg>, <Angle 11.08379885 deg>,
       <Angle 11.32672395 deg>, <Angle 11.32681116 deg>,
       <Angle 11.38157109 deg>, <Angle 11.618352

In [68]:
tracts_n708[np.argsort(distance)]

array([ 9813,  9327,  9328,  9326,  9329,  9568,  9085,  9325,  9084,
        9086,  9330,  9083,  9087,  9324,  9331, 10059,  9082,  9088,
        9323,  9332,  9081,  9089,  9565,  9322,  9080,  9090,  9333,
        9564,  9321,  9079,  9091,  9334,  9563,  9078,  9092,  9320,
        9335,  9562,  9077,  9319,  9336,  9561,  9076,  9318,  9560,
        9075,  9317,  9074,  9096,  9559,  9316,  9339,  9073,  9097,
        9558,  9315,  9340, 10284,  9072,  9098,  9557,  9314,  9341,
        9071,  9099,  9556,  9313,  9342,  9100,  9586,  9343,  9101,
        9587,  9344,  9102,  9345,  9103,  9346,  9104,  9347,  9836,
        9594,  9373,  9131,  9374,  9132,  9375,  9133,  9376,  9134,
        9377,  9135,  9378,  9136,  9379,  8286,  9011,  8769,  8527,
        8285,  9010,  8768,  8526,  8284,  8767,  8525,  8283,  8766,
        8524,  8282,  8765,  8523,  8281,  8764,  8522,  8280,  8521,
        9471,  9470,  9227,  9469,  9226,  9468,  9467,  9466,  9465,
        9459,  9458,

----
# Want to find patches that have been reduced for Merian but have not yet been processed with GaAP

In [22]:
def findGaapReducedPatches(tract, repo = '/scratch/gpfs/am2907/Merian/gaap/'):
    dir = os.path.join(repo, f"log/{tract}/")
    logs = np.array(glob.glob(dir + "*.o"))
    logs_patches = np.array([log.split("/")[-1].split(".")[0] for log in logs]).astype(int)
    logs, logs_patches = logs[logs_patches.argsort()], logs_patches[logs_patches.argsort()]

    return (logs_patches)

In [23]:
tracts_n708_nogaap = []
for tract in tracts:
    patches_mer  = findReducedPatches(tract)
    patches_gaap = findGaapReducedPatches(tract)
    if len(set(patches_mer) - set(patches_gaap)) > 0:
        tracts_n708_nogaap.append(tract)

In [9]:
tracts_n708_gaap = []
for tract in tracts:
    patches_gaap = findGaapReducedPatches(tract)
    if len(patches_gaap) > 0:
        tracts_n708_gaap.append(tract)

In [24]:
print(tracts_n708_nogaap)

[9617, 9618, 9619, 9620, 9621, 9697, 9698, 9699, 9700, 9701, 9702, 9703, 9707, 9708, 9709, 9710, 9711, 9712, 9713, 9714, 9798, 9799, 9800, 9801, 9802, 9803, 9804, 9805, 9806, 9807, 9808, 9809, 9810, 9811, 9812, 9814, 9815, 9816, 9817, 9818, 9819, 9820, 9821, 9828, 9833, 9837, 9838, 9839, 9862, 9863, 9939, 9940, 9941, 9942, 9943, 9944, 9945, 9949, 9950, 9951, 9952, 9953, 10040, 10041, 10042, 10043, 10044, 10045, 10046, 10047, 10048, 10049, 10050, 10051, 10052, 10053, 10055, 10057, 10058, 10060, 10061, 10062, 10070, 10078, 10182, 10183, 10184, 10185, 10186, 10283, 10285, 10286, 10287, 10288, 10289, 10290, 10291, 10292, 10293, 10294, 10295, 10296, 10297, 10298, 10299, 10300, 10301, 10302, 10303, 10304, 10426, 10427, 10428]


In [14]:
tract = 9573
patches_mer  = findReducedPatches(tract)
patches_gaap = findGaapReducedPatches(tract)

In [15]:
len(patches_mer), len(patches_gaap)

(79, 0)

In [47]:
for tract in tracts_n708_nogaap:
    patches  = list(set(findReducedPatches(tract))- set(findGaapReducedPatches(tract)))
    if len(patches) > 0:
        print (f'TRACT:{tract}, {len(patches)}')


TRACT:9566, 81
TRACT:9567, 81
TRACT:9568, 79
TRACT:9569, 80
TRACT:9570, 77
TRACT:9571, 80
TRACT:9572, 77
TRACT:9573, 79
TRACT:9574, 80
TRACT:9575, 81
TRACT:9576, 80
TRACT:9577, 81
TRACT:9578, 74
TRACT:9579, 62
TRACT:9580, 2
TRACT:9582, 4
TRACT:9583, 5
TRACT:9584, 7
TRACT:9585, 7
TRACT:9587, 47
TRACT:9588, 53
TRACT:9589, 52
TRACT:9590, 48
TRACT:9591, 49
TRACT:9592, 49
TRACT:9593, 50
TRACT:9594, 48
TRACT:9595, 44
TRACT:9596, 27


In [58]:
print(tracts_n708_nogaap)

[9566, 9567, 9568, 9569, 9570, 9571, 9572, 9573, 9574, 9575, 9576, 9577, 9578, 9579, 9580, 9582, 9583, 9584, 9585, 9587, 9588, 9589, 9590, 9591, 9592, 9593, 9594, 9595, 9596]
